In [273]:
import numpy as np
from scipy.linalg import solve

In [274]:
np.random.seed(0)

### 3. Next step solution

In [275]:
def interior_point_step(A, b, x, y, s, mu):
    n = x.shape[0]  # variables
    e = np.ones_like(x)

    # Diagonal matrices
    X = np.diag(x)
    S_inv = np.diag(1 / s)

    # Short-step update for mu
    mu_prime = (1 - (1 / (6 * np.sqrt(n)))) * mu

    # Solve (A S^{-1} X A^T) k = b - mu' * A S^{-1} e
    M = A @ S_inv @ X @ A.T
    rhs = b - mu_prime * A @ S_inv @ e
    dy = np.linalg.solve(M, rhs) 

    ds = -A.T @ dy
    dx = -X @ S_inv @ ds + mu_prime * S_inv @ e - x

    # Update
    x_prime = x + dx
    y_prime = y + dy
    s_prime = s + ds

    return x_prime, y_prime, s_prime, mu_prime


In [ ]:
def mehrotra_step(A, b, c, x, y, s, mu, tol=1e-10):
    n = x.size
    e = np.ones_like(x)

    # Residuals (correct signs)
    r_p = A @ x - b            # primal residual
    r_d = A.T @ y + s - c      # dual residual
    r_c = x * s                # complementarity residual (element-wise)

    # Form diagonal matrices efficiently
    X_inv = 1.0 / x
    XS_inv = np.diag(X_inv * s)  # Diagonal matrix with s_i / x_i

    # Schur complement matrix
    schur = A @ XS_inv @ A.T

    # Affine scaling direction (predictor)
    rhs_aff = r_p - A @ (X_inv * (r_c - s * r_d))
    dy_aff = np.linalg.solve(schur, rhs_aff)
    ds_aff = r_d - A.T @ dy_aff
    dx_aff = X_inv * (-r_c - s * ds_aff)

    # Compute max step length to maintain positivity
    def max_step(x, dx):
        idx = dx < 0
        if np.any(idx):
            return min(1.0, 0.99 * np.min(-x[idx] / dx[idx]))
        return 1.0

    alpha_aff_primal = max_step(x, dx_aff)
    alpha_aff_dual = max_step(s, ds_aff)

    # Affine duality measure
    x_aff = x + alpha_aff_primal * dx_aff
    s_aff = s + alpha_aff_dual * ds_aff
    mu_aff = np.dot(x_aff, s_aff) / n

    # Centering parameter
    sigma = (mu_aff / mu) ** 3

    # Corrector residual for complementarity
    r_c_corr = r_c + dx_aff * ds_aff - sigma * mu * e
    rhs_corr = r_p - A @ (X_inv * (r_c_corr - s * r_d))
    dy = np.linalg.solve(schur, rhs_corr)
    ds = r_d - A.T @ dy
    dx = X_inv * (-r_c_corr - s * ds)

    # Step length for full step
    alpha_primal = max_step(x, dx)
    alpha_dual = max_step(s, ds)
    alpha = min(alpha_primal, alpha_dual)

    # Update variables with step length
    x_new = x + alpha * dx
    y_new = y + alpha * dy
    s_new = s + alpha * ds

    mu_new = np.dot(x_new, s_new) / n

    return x_new, y_new, s_new, mu_new


In [278]:
np.random.seed(0)
m, n = 2, 4
A = np.random.rand(m, n)
b = np.random.rand(m)
c = np.random.rand(n)

# Strictly feasible starting point
x0 = np.random.rand(n) + 1.0
y0 = np.random.rand(m)
s0 = np.random.rand(n) + 1.0
mu0 = (x0 @ s0) / n

In [279]:
interior_point_step(A, b, x0, y0, s0, mu0)

(array([ 0.73163932,  0.53785387,  1.02813365, -0.81165922]),
 array([ 2.02530982, -2.01437071]),
 array([2.51614654, 2.77021386, 1.97191022, 3.6731912 ]),
 np.float64(2.0233470621836656))

### 6. Show that vectors are strictly feasiblefor X and its dual

In [280]:
c = np.array([-3, -4, 0, 0, 0])

A = np.array([[3, 3, 3, 0, 0],
              [3, 1, 0, 1, 0],
              [1, 4, 0, 0, 1]])

b = np.array([4, 3, 4])

In [281]:
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15])
y = np.array([-4/5, -4/5, -2/3])
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3])

In [282]:
#  x > 0
print("X is strictily positive:", all(x > 0)) 

# Ax = b
print("Ax = b:", np.allclose(A @ x, b, atol=1e-10))

# s > 0
print("s is strictly positive:", all(s > 0))

# A^T y + s = c
print("A^T y + s = c:", np.allclose(A.T @ y + s, c, atol=1e-10))

print("x, y and s are strictly feasible solutions of both problem X and its dual")

X is strictily positive: True
Ax = b: True
s is strictly positive: True
A^T y + s = c: True
x, y and s are strictly feasible solutions of both problem X and its dual


### 7. Show x,y,s are good starting solution for P_mu

In [283]:
n = x.shape[0]
mu = (x @ s) / n
mu

np.float64(0.9866666666666667)

In [284]:
# Complementary slackness
for xi, si in zip(x, s):
    print(f"x_i * s_i = {xi * si:.4f} (should be close to mu = {mu:.4f})")
print("The vectors are a good starting solution")
print("Since mu is close to 1, the choice for 1 would be a good initial guess.")

x_i * s_i = 0.9867 (should be close to mu = 0.9867)
x_i * s_i = 0.9956 (should be close to mu = 0.9867)
x_i * s_i = 0.9600 (should be close to mu = 0.9867)
x_i * s_i = 1.0133 (should be close to mu = 0.9867)
x_i * s_i = 0.9778 (should be close to mu = 0.9867)
The vectors are a good starting solution
Since mu is close to 1, the choice for 1 would be a good initial guess.


### 8. Iterate next-step to convergence

In [300]:
count = 0

_x, _y, _s, _mu = x.copy(), y.copy(), s.copy(), mu
while True:
    x_prime, y_prime, s_prime, mu_prime = interior_point_step_mehrotra(A, b, c, _x, _y, _s, _mu)
    # x_fixed, y_fixed, s_fixed, feasible, optimal = verify(A, b, c, x_prime, y_prime, s_prime, mu_prime)
    
    # if feasible and optimal:
    #     _x, _y, _s = x_fixed, y_fixed, s_fixed
    #     break
    print(np.linalg.norm(x_prime - _x), np.linalg.norm(y_prime - _y), np.linalg.norm(s_prime - _s))
    
    count += 1

    if np.allclose(_x, x_prime, atol=1e-15) and np.allclose(_y, y_prime, atol=1e-15) and np.allclose(_s, s_prime, atol=1e-15):
        break
    
    _x, _y, _s, _mu = x_prime, y_prime, s_prime, mu_prime
    
print("x", _x)
print("y", _y)
print("s", _s)
print("mu", _mu)
print(f"Converged in {count} iterations")

301.12408367926804 35.13370059886382 217.18210613104924
9099065.244775 3233851.0311050396 15211294.148351682
5.787018856429829e+17 6.6857392959449624e+16 4.1277666663143974e+17
7.012637644657352e+38 2.4924023794174557e+38 1.1723531499247773e+39
2.1135371778377812e+71 2.4417681907777065e+70 1.5075444761727132e+71
4.653137711580487e+102 1.653798768989005e+102 7.778985496792617e+102
1.0350953092021739e+135 1.195844968773515e+134 7.383131141778254e+134
inf inf inf
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan

C:\Users\sebas\AppData\Local\Temp\ipykernel_24024\1597348453.py:14: RuntimeWarning: overflow encountered in matmul
  r_c = X @ s
C:\Users\sebas\AppData\Local\Temp\ipykernel_24024\1597348453.py:20: RuntimeWarning: invalid value encountered in matmul
  rhs_aff = r_p - A @ (X_inv @ (r_c - S @ (-r_d)))


nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan 

KeyboardInterrupt: 

In [286]:
count = 0

_x, _y, _s, _mu = x.copy(), y.copy(), s.copy(), mu

while True:
    x_prime, y_prime, s_prime, mu_prime = interior_point_step(A, b, _x, _y, _s, _mu)
    count += 1

    if np.allclose(_x, x_prime, atol=1e-15) and np.allclose(_y, y_prime, atol=1e-15) and np.allclose(_s, s_prime, atol=1e-15):
        break
    
    _x, _y, _s, _mu = x_prime, y_prime, s_prime, mu_prime
    
print("x", x)
print("y", y)
print("s", s)
print("mu", _mu) 
print(f"Converged in {count} iterations")

x [0.4        0.53333333 0.4        1.26666667 1.46666667]
y [-0.8        -0.8        -0.66666667]
s [2.46666667 1.86666667 2.4        0.8        0.66666667]
mu 4.263535721893573e-15
Converged in 428 iterations


In [287]:
# Check convergence
for xi, si in zip(x, s):
    print(f"x_i * s_i = {xi * si:.15f} (should be close to mu = {mu:.15f})")

x_i * s_i = 0.986666666666667 (should be close to mu = 0.986666666666667)
x_i * s_i = 0.995555555555556 (should be close to mu = 0.986666666666667)
x_i * s_i = 0.960000000000000 (should be close to mu = 0.986666666666667)
x_i * s_i = 1.013333333333333 (should be close to mu = 0.986666666666667)
x_i * s_i = 0.977777777777778 (should be close to mu = 0.986666666666667)
